<a href="https://colab.research.google.com/github/Neofly4023/bso-fs/blob/master/bso_fs_timeoptim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies
This part is in case you run the book in local, because Colab environment already contains most of popular python packages for ML ( like numpy, pandas and sklearn )

In [0]:
!pip install xlsxwriter pandas numpy sklearn

# Import dependencies
This par contains all the dependencies that you need to import to run the notebook

In [0]:
import pandas as pd
import numpy as np
import os, glob, random, copy, sys, time, re
from collections import defaultdict
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, \
ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from oauth2client.client import GoogleCredentials
from google.colab import drive
import xlsxwriter

# Mounting your drive 
This part is to mount the drive and be able to save results in it

In [0]:
gdrive_path = '/content/gdrive'
drive.mount(gdrive_path)

# Bee class
This class is the implementation of the Bee Swarm Optimization bee

In [0]:
class Bee :
    def __init__(self,id,problem,locIterations,state):
        self.id=id
        self.data=problem
        self.solution = Solution(self.data,state=state)
        self.fitness= 0.0
        self.rl_return = 0.0
        self.locIterations=locIterations
        self.action = []
    
    def localSearch(self):
        best=self.fitness
        #done=False
        lista=[j for j, n in enumerate(self.solution.state) if n == 1]
        indice =lista[0]
        
        for itr in range(self.locIterations):
            while(True):
                pos=-1
                oldFitness=self.fitness
                for i in range(len(self.solution.state)):
                    
                    if ((len(lista)==1) and (indice==i) and (i < self.data.nb_attribs-1)):
                        i+=1
                    self.solution.state[i]= (self.solution.state[i] + 1) % 2
                    
                    quality = self.solution.get_accuracy(self.solution.get_state())
                    if (quality >best):
                        pos = i
                        best=quality
                    self.solution.state[i]= (self.solution.state[i]+1) % 2
                    self.fitness = oldFitness 
                if (pos != -1):
                    self.solution.state[pos]= (self.solution.state[pos]+1)%2
                    self.fitness = best
                else:
                    break
            for i in range(len(self.solution.state)):
                oldFitness=self.fitness
                if ((len(lista)==1) and (indice==i) and (i < self.data.nb_attribs-1)):
                    i+=1
                self.solution.state[i]= (self.solution.state[i] + 1) % 2
                quality = self.solution.get_accuracy(self.solution.get_state())
                if (quality<best):
                    self.solution.state[i]= (self.solution.state[i] + 1) % 2
                    self.fitness = oldFitness


    def ql_localSearch(self,maxIterIndex):
      
        """The reason why we devide the loc by max is to favor exploitation 
        when loc > max and exploration when loc < max """
        
        iterations = int(maxIterIndex/self.locIterations) if self.locIterations <= maxIterIndex else 1
        for itr in range(iterations):
       
            state = self.solution.get_state()
            #next_state, action = self.data.ql.step(self.solution,self.data.nb_attribs)
            
            # Ths first +1, is not to devide by 0, the 2nd one, is not to get an empty list in case iterations > nb_atts
            next_state, action = self.data.ql.step(self.solution,int(self.data.nb_attribs/(iterations+1))+1)
            next_sol = Solution(self.data,state=next_state)
            acc_state = self.solution.get_accuracy(state)
            acc_new_state = self.solution.get_accuracy(next_state)

            if (acc_state < acc_new_state):
                reward = acc_new_state
            elif (acc_state > acc_new_state):
                reward = acc_new_state - acc_state
            else :
                if (Solution.nbrUn(state) > Solution.nbrUn(next_state) ):
                    reward = 1/2 * acc_new_state
                else :
                    reward = -1/2 * acc_new_state
            
            self.data.ql.learn(self.solution,action,reward,next_sol)
            self.solution = next_sol
            self.rl_return = self.data.ql.get_q_value(self.solution,action)
            self.fitness = self.solution.get_accuracy(self.solution.get_state())
            
    
    def setSolution(self,solution):
        self.solution.set_state(solution)
        self.fitness = self.solution.get_accuracy(solution)
    
    @classmethod
    def Rand(self, num, start=None, end=None): 
        res = [] 
        if (not start) or (not end): 
          """We used 20%/80% (Pareto's law) to initilize the solution"""
          res = np.random.choice([0,1],size=(num,),p=[2./10,8./10]).tolist()
        else: 
          for j in range(num): 
              res.append(random.randint(start, end))
        return res 

# Swarm class
This class is the implementation of the Bee Swarm Optimization swarm

In [0]:
class Swarm :
    def __init__(self,problem,flip,maxChance,nbrBees,maxIterations,locIterations):
        self.data=problem
        self.flip=flip
        self.maxChance=maxChance
        self.nbChance=maxChance
        self.nbrBees=nbrBees
        self.maxIterations=maxIterations
        self.locIterations=locIterations
        self.beeList=[]
        self.refSolution = Bee(-1,self.data,self.locIterations,Bee.Rand(self.data.nb_attribs))
        self.bestSolution = self.refSolution
        self.tabou=[]

    def searchArea(self):    
        i=0
        h=0
        
        self.beeList=[]
        while((i<self.nbrBees) and (i < self.flip) ) :
            #print ("First method to generate")
            
            solution=self.refSolution.solution.get_state()
            k=0
            while((self.flip*k+h) < len(solution)):
                solution[self.flip*k +h] = ((solution[self.flip*k+h]+1) % 2)
                k+=1
            newBee=Bee(i,self.data,self.locIterations,solution)
            self.beeList.append(newBee)
            
            i+=1
            h=h+1
        h=0
        
        while((i<self.nbrBees) and (i< 2*self.flip )):
            #print("Second method to generate")

            solution=self.refSolution.solution.get_state()
            k=0
            while((k<int(len(solution)/self.flip)) and (self.flip*k+h < len(solution))):
                solution[int(self.data.nb_attribs/self.flip)*h+k] = ((solution[int(self.data.nb_attribs/self.flip)*h+k]+1)%2)
                k+=1
            newBee=Bee(i,self.data,self.locIterations,solution)
            self.beeList.append(newBee)
            
            i+=1
            h=h+1
        while (i<self.nbrBees):
            #print("Random method to generate")
            solution= self.refSolution.solution.get_state()
            indice = random.randint(0,len(solution)-1)
            solution[indice]=((solution[indice]+1) % 2)
            newBee=Bee(i,self.data,self.locIterations,solution)
            self.beeList.append(newBee)
            i+=1
        for bee in (self.beeList):
            lista=[j for j, n in enumerate(bee.solution.get_state()) if n == 1]
            if (len(lista)== 0):
                bee.setSolution(Bee.Rand(self.data.nb_attribs))
                
    def selectRefSol(self):
      self.beeList.sort(key=lambda Bee: Bee.fitness, reverse=True)
      bestQuality=self.beeList[0].fitness
      if(bestQuality>self.bestSolution.fitness):
          self.bestSolution=self.beeList[0]
          self.nbChance=self.maxChance
          return self.bestSolution
      else:
          if(  (len(self.tabou)!=0) and  bestQuality > (self.tabou[len(self.tabou)-1].fitness)):
              self.nbChance=self.maxChance
              return self.bestBeeQuality()
          else:
              self.nbChance-=1
              if(self.nbChance > 0): 
                  return self.bestBeeQuality()
              else :
                  return self.bestBeeDiversity()
      
    def distanceTabou(self,bee):
        distanceMin=self.data.nb_attribs
        for i in range(len(self.tabou)):
            cpt=0
            for j in range(self.data.nb_attribs):
                if (bee.solution.get_state()[j] != self.tabou[i].solution.get_state()[j]) :
                      cpt +=1
            if (cpt<=1) :
                return 0
            if (cpt < distanceMin) :
                distanceMin=cpt
        return distanceMin
    
    def bestBeeQuality(self):
        distance = 0
        i=0
        pos=-1
        while(i<self.nbrBees):
            max_val=self.beeList[i].fitness
            nbUn=Solution.nbrUn(self.beeList[i].solution.get_state())
            while((i<self.nbrBees) and (self.beeList[i].solution.get_accuracy(self.beeList[i].solution.get_state()) == max_val)):
                distanceTemp=self.distanceTabou(self.beeList[i])
                nbUnTemp = Solution.nbrUn(self.beeList[i].solution.get_state())
                if(distanceTemp > distance) or ((distanceTemp == distance) and (nbUnTemp < nbUn)):
                    if((distanceTemp==distance) and (nbUnTemp<nbUn)):
                        print("We pick the solution with less features")
                    nbUn=nbUnTemp
                    distance=distanceTemp
                    pos=i
                i+=1
            if(pos!=-1) :
                return self.beeList[pos]
        bee= Bee(-1,self.data,self.locIterations,Bee.Rand(self.data.nb_attribs))
        return bee
            
    def bestBeeDiversity(self):
        max_val=0
        for i in range(len(self.beeList)):
            if (self.distanceTabou(self.beeList[i])> max_val) :
                max_val = self.distanceTabou(self.beeList[i])
        if (max_val==0):
            bee= Bee(-1,self.data,self.locIterations,Bee.Rand(self.data.nb_attribs))
            return bee
        i=0
        while(i<len(self.beeList) and self.distanceTabou(self.beeList[i])!= max_val) :
            i+=1
        return self.beeList[i]
    
    def bso(self,typeOfAlgo):
        i=1
        while(i<=self.maxIterations):
            #print("refSolution is : ",self.refSolution.solution)
            self.tabou.append(self.refSolution)
            #print("Iteration N° : ",i)
            
            self.searchArea()

            # The local search part
            
            for j in range(self.nbrBees):
              if (typeOfAlgo == 0):
                self.beeList[j].localSearch()
              elif (typeOfAlgo == 1):
                for episode in range(self.maxIterations):
                  self.beeList[j].ql_localSearch(i)
                #print( "Q-value of bee " + str(j) + " solution is : " + str(self.beeList[j].fitness))
            self.refSolution = self.selectRefSol()
            i+=1
            
        print("[BSO parameters used]\n")
        print("Type of algo : {0}".format(typeOfAlgo))
        print("Flip : {0}".format(self.flip))
        print("MaxChance : {0}".format(self.maxChance))
        print("Nbr of Bees : {0}".format(self.nbrBees))
        print("Nbr of Max Iterations : {0}".format(self.maxIterations))
        print("Nbr of Loc Iterations : {0}\n".format(self.locIterations))
        print("Best solution found : ",self.bestSolution.solution.get_state())
        print("Number of features used : {0}".format(Solution.nbrUn(self.bestSolution.solution.get_state())))
        print("Accuracy : {0:.2f} ".format(self.bestSolution.fitness*100))

        if (typeOfAlgo == 1):
          print("Return (Q-value) : ",self.bestSolution.rl_return)
          
        return self.bestSolution.fitness*100, Solution.nbrUn(self.bestSolution.solution.get_state())

    
    def str_sol(self,mlist):
        result = ''
        for element in mlist:
            result += str(element)
        return result

# Solution class
This class is the implementation of a solution, it contains attributs that a solution could have ( like the subset of featuresor the evaluation, to not train a model with the same subset of features each time ), it could be extensible to other attributs

In [0]:
class Solution:

    solutions = {} 

    def __init__(self,data,state,accuracy=0):
        self.data = data
        self.accuracy = accuracy
        self.state = state
        self.solutions[Solution.str_sol(self.state)] = self 

    def get_accuracy(self,state):
        if (Solution.str_sol(state) in Solution.solutions):
            if (Solution.solutions[Solution.str_sol(state)].accuracy == 0) :
                Solution.solutions[Solution.str_sol(state)].set_accuracy(state)
        else :
            Solution.solutions[Solution.str_sol(state)] = Solution(self.data,state=state)
            Solution.solutions[Solution.str_sol(state)].set_accuracy(state)

        return Solution.solutions[Solution.str_sol(state)].accuracy

    def get_state(self):
        return self.state.copy()

    def set_accuracy(self,state):
        Solution.solutions[Solution.str_sol(state)].accuracy = self.data.evaluate(state)       
    
    def set_state(self,state):
        if (Solution.str_sol(state) in Solution.solutions): 
            Solution.solutions[Solution.str_sol(state)].state = state.copy()
        else :
            Solution.solutions[Solution.str_sol(state)] = Solution(self.data,state=state)

    @staticmethod
    def str_sol(mlist):
        result = ''
        for element in mlist:
            result += str(element)
        return result

    @staticmethod
    def nbrUn(state):
        return len([i for i, n in enumerate(state) if n == 1])

# Reinforcement learning class
This class containts the implementation of the different RL algorithms

In [0]:
class QLearning:
    def __init__(self,nb_atts,actions):
        self.actions = actions
        self.alpha = 0.1
        self.gamma = 0.9
        self.epsilon = 0.1
        self.q_table = [ {} for i in range(nb_atts) ] 

    def get_max_q_value(self,solution,actions_vals):
        max_val = 0.0
        arg_max = 0

        for i in actions_vals:
            state_i = self.get_next_state(solution,i)
            state_i_acc = solution.get_accuracy(state_i)
            if state_i_acc > max_val:
                max_val = self.get_q_value(solution,i) + state_i_acc
                arg_max = i

        return max_val, arg_max


    def get_q_value(self,solution,action):
        
        state = solution.get_state()
        if not Solution.str_sol(state) in self.q_table[Solution.nbrUn(state)]:
            self.q_table[Solution.nbrUn(state)][Solution.str_sol(state)] = {}

        if not str(action) in self.q_table[Solution.nbrUn(state)][Solution.str_sol(state)]:
            self.q_table[Solution.nbrUn(state)][Solution.str_sol(state)][str(action)] = solution.get_accuracy(self.get_next_state(solution,action))
            
        return self.q_table[Solution.nbrUn(state)][Solution.str_sol(state)][str(action)]

    
    def set_q_value(self,solution,action,val):
        state = solution.get_state()
        self.q_table[Solution.nbrUn(state)][Solution.str_sol(state)][str(action)] = val

    
    def step(self,solution,sample_size):
        if np.random.uniform() > self.epsilon :
            
            action_values = [self.actions[i] for i in sorted(random.sample(range(len(self.actions)), sample_size))]
            
            max_val = self.get_max_q_value(solution,action_values)[0]
            argmax_actions=[self.get_max_q_value(solution,action_values)[1]] 

            for ac in action_values :
              ac_state = self.get_next_state(solution,ac)
              ac_state_q_val = self.get_q_value(solution,ac) + solution.get_accuracy(ac_state)
              
              if ( ac_state_q_val >= max_val ):
                  argmax_actions.append(ac)

            next_action = np.random.choice(argmax_actions)
            next_state = self.get_next_state(solution,next_action)

        else :
            next_action = np.random.choice(self.actions)
            next_state = self.get_next_state(solution,next_action)

        if self.epsilon > 0 :
            self.epsilon -= 0.0001 
        if self.epsilon < 0 :
            self.epsilon = 0

        return next_state, next_action 


    def get_next_state(self,solution,action):
        next_state = solution.get_state()
        next_state[action] = (next_state[action]+1) % 2
        if (Solution.nbrUn(next_state) != 0):
          return next_state
        else:
          return solution.get_state()
    
    def learn(self,current_sol,current_action,reward,next_sol):
        next_action = self.get_max_q_value(next_sol,self.actions)[1]
        new_q = reward + self.gamma * self.get_q_value(next_sol,next_action)  
        self.set_q_value(current_sol,current_action,(1 - self.alpha)*self.get_q_value(current_sol,current_action) + self.alpha*new_q)  

# Evaluation class
This class contains the implementation of evaluation methods and could be extended to other problems than feature selection

In [0]:
class FsProblem :
    def __init__(self,typeOfAlgo,data,qlearn,classifier=KNeighborsClassifier(n_neighbors=1)):
        self.data=data
        self.nb_attribs = len(self.data.columns)-1 
        self.outPuts=self.data.iloc[:,self.nb_attribs]
        self.ql = qlearn
        self.nb_actions = len(self.ql.actions)
        self.classifier = classifier
        self.typeOfAlgo = typeOfAlgo

    def evaluate2(self,solution):
        sol_list = self.sol_to_list(solution)
        if (len(sol_list) == 0):
            return 0
         
        df = self.data.iloc[:,sol_list]
        array=df.values
        X = array[:,0:self.nb_attribs]
        Y = self.outPuts
        train_X, test_X, train_y, test_y = train_test_split(X, Y, 
                                                    random_state=0,
                                                    test_size=0.1
                                                    )
        self.classifier.fit(train_X,train_y)
        predict= self.classifier.predict(test_X) 
        return metrics.accuracy_score(predict,test_y)


    def evaluate(self,solution):
        sol_list = self.sol_to_list(solution)
        if (len(sol_list) == 0):
            return 0
        
        df = self.data.iloc[:,sol_list]        
        array=df.values
        X = array[:, 0:self.nb_attribs]
        Y = self.outPuts
        cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
        results = cross_val_score(self.classifier, X, Y, cv=cv,scoring='accuracy')
        #print("\n[Cross validation results]\n{0}".format(results))
        return results.mean()

    def sol_to_list(self,solution):
        sol_list=[i for i, n in enumerate(solution) if n == 1]
        return sol_list

# Data class
This class is the one resbonsible for processing and getting the data ready to be trained

In [0]:
class FSData():

    def __init__(self,typeOfAlgo,location,nbr_exec):
        
        self.typeOfAlgo = typeOfAlgo
        self.location = location
        self.nb_exec = nbr_exec
        self.dataset_name = re.search('[A-Za-z\-]*.csv',self.location)[0].split('.')[0]
        self.df = pd.read_csv(self.location,header=None)
        self.ql = QLearning(len(self.df.columns),self.attributs_to_flip(len(self.df.columns)-1))
        self.fsd = FsProblem(self.typeOfAlgo,self.df,self.ql)
        
        self.classifier_name = str(type(self.fsd.classifier)).strip('< > \' class ').split('.')[3]
        path = gdrive_path + '/My Drive/Colab Notebooks/results/'+ self.dataset_name
        if not os.path.exists(path):
          os.makedirs(path + '/logs/')
          os.makedirs(path + '/sheets/')
        self.instance_name = str(self.typeOfAlgo) + '-' + str(time.strftime("%m-%d-%Y_%H-%M-%S_", time.localtime()) + self.classifier_name)
        log_filename = str(path + '/logs/'+ self.instance_name)
        if not os.path.exists(path):
          os.makedirs(path)
        log_file = open(log_filename + '.txt','w+')
        sys.stdout = log_file
        
        print("[START] Dataset" + self.dataset_name + "description \n")
        print("Shape : " + str(self.df.shape) + "\n")
        print(self.df.describe())
        print("\n[END] Dataset" + self.dataset_name + "description\n")
        print("[START] Ressources specifications\n")
        !cat /proc/cpuinfo # Think of changing this when using Windows
        print("[END] Ressources specifications\n")

        
        sheet_filename = str(path + '/sheets/'+ self.instance_name )
        self.workbook = xlsxwriter.Workbook(sheet_filename + '.xlsx')
        
        self.worksheet = self.workbook.add_worksheet(self.classifier_name)
        self.worksheet.write(0,0,'Iteration')
        self.worksheet.write(0,1,'Accuracy')
        self.worksheet.write(0,2,'N_Features')
        self.worksheet.write(0,3,'Time')
    
    
    def attributs_to_flip(self,nb_att):

        return list(range(nb_att))
    
    def run(self,flip,maxChance,nbrBees,maxIterations,locIterations):
        total_time = 0
        
        for itr in range(1,self.nb_exec+1):
          print ("Execution {0}".format(str(itr)))
          self.fsd = FsProblem(self.typeOfAlgo,self.df,self.ql)
          swarm = Swarm(self.fsd,flip,maxChance,nbrBees,maxIterations,locIterations)
          t1 = time.time()
          best = swarm.bso(self.typeOfAlgo)
          t2 = time.time()
          total_time += t2-t1
          print("Time elapsed for execution {0} : {1:.2f} s\n".format(itr,t2-t1))
          self.worksheet.write(itr, 0, itr)
          self.worksheet.write(itr, 1, "{0:.2f}".format(best[0]))
          self.worksheet.write(itr, 2, best[1])
          self.worksheet.write(itr, 3, "{0:.3f}".format(t2-t1))
          
        print ("Total execution time of {0} executions \nfor dataset \"{1}\" is {2:.2f} s".format(self.nb_exec,self.dataset_name,total_time))
        self.workbook.close()


# The main program
In the part you can specify the parameters' values.
Those parameters are :


*   **dataset** : the dataset name ( without **.cvs** extension ), from the list of [datasets](https://github.com/Neofly4023/bso-fs/tree/master/datasets), or you can put your own **.csv** dataset (without header row, and index column)
*   **typeOfAlgo** : for now, the value is 0, for the original localsearch algorithm, or 1, for the q-localsearch algorithm
*  **nbr_exec** : the number of executions
*  **flip** : the flip parameter is used to generate solutions from the searchArea
*  **maxChance** : the maxChance parameter is a used for exploitation
*  **nbrBees** : the number of Bees used
*  **maxIterations** : the number of iterations inside the BSO algorithm, it is possible to not reach it
*  **locIterations** : the number of iterations inside the localsearch algorithms (orginal & q-learning)

For more details, please check the previous [work](https://link.springer.com/chapter/10.1007%2F978-3-319-19258-1_33), or PM me via : [e-mail](mailto:ea_remache@esi.dz)

In [0]:
# Main program

# Prepare the dataset

dataset = "BreastEW"
data_loc_path = "https://raw.githubusercontent.com/Neofly4023/bso-fs/master/datasets/"
location = data_loc_path + dataset + ".csv"

# Params init

typeOfAlgo = 1
nbr_exec = 1
flip = 5
maxChance = 3
nbrBees = 10
maxIterations = 4
locIterations = 2

instance = FSData(typeOfAlgo,location,nbr_exec)
instance.run(flip,maxChance,nbrBees,maxIterations,locIterations)